In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
df = pd.read_csv('./processed/processed_sa_bert_price_diff.csv', index_col=[0])
df

,positive,negative,neutral,none,count,known,to_predict
Date,,,,,,,
2018-01-03,22,11,1,0,34,0.37,-0.190000
2018-01-04,24,8,4,0,36,-0.19,-0.040000
2018-01-05,21,5,2,0,28,-0.04,0.010000
2018-01-08,24,1,1,0,26,0.01,-0.140000
2018-01-09,25,8,5,0,38,-0.14,-0.140000
...,...,...,...,...,...,...,...
2018-05-24,14,5,3,0,22,-0.17,-0.040000
2018-05-25,14,3,1,0,18,-0.04,0.120000
2018-05-29,20,5,2,0,27,0.12,0.010000


In [3]:
processed_absa_price_diff = pd.read_csv(os.path.join('processed','processed_absa_price_diff.csv'),delimiter=',',index_col=0)
processed_day_level_absa = pd.read_csv(os.path.join('processed','processed_day_level_absa.csv'),delimiter=',',index_col=0)
processed_day_level_sa_roberta = pd.read_csv(os.path.join('processed','processed_day_level_sa_roberta.csv'),delimiter=',',index_col=0)
processed_day_level_sa_bert = pd.read_csv(os.path.join('processed','processed_day_level_sa_bert.csv'),delimiter=',',index_col=0)
processed_sa_bert_price_diff = pd.read_csv(os.path.join('processed','processed_sa_bert_price_diff.csv'),delimiter=',', index_col=0)
processed_sa_roberta_price_diff = pd.read_csv(os.path.join('processed','processed_sa_roberta_price_diff.csv'),delimiter=',',index_col=0)


In [6]:
X = processed_sa_roberta_price_diff[['positive', 'negative', 'neutral', 'none', 'known']].values
y = processed_sa_roberta_price_diff[['to_predict']].values

In [5]:
X = processed_sa_bert_price_diff[['known']].values
y = processed_sa_bert_price_diff[['to_predict']].values

In [ ]:
filtered_ABSA = 'usability_positive'
if filtered_ABSA == 'usability_positive':
    X = processed_absa_price_diff[['known','usability_positive']].values
elif filtered_ABSA == 'design features_negative':
    X = processed_absa_price_diff[['known','design features_negative']].values
elif filtered_ABSA == 'general_neutral':
    X = processed_absa_price_diff[['known','general_neutral']].values
elif filtered_ABSA == 'price_positive':
    X = processed_absa_price_diff[['known','price_positive']].values
elif filtered_ABSA == 'quality_positive':
    X = processed_absa_price_diff[['known','quality_positive']].values
elif filtered_ABSA == 'operation performance_negative':
    X = processed_absa_price_diff[['known','operation performance_negative']].values
elif filtered_ABSA == 'operation performance_positive':
    X = processed_absa_price_diff[['known','operation performance_positive']].values
elif filtered_ABSA == 'general_negative':
    X = processed_absa_price_diff[['known','general_negative']].values
elif filtered_ABSA == 'design features_positive':
    X = processed_absa_price_diff[['known','design features_positive']].values
elif filtered_ABSA == 'general_positive':
    X = processed_absa_price_diff[['known','general_positive']].values
elif filtered_ABSA == 'quality_negative':
    X = processed_absa_price_diff[['known','quality_negative']].values
X = scaler.fit_transform(X.reshape(-1,2))

In [59]:
def get_absa_df():
    absa_df = pd.read_csv('./data/processed_absa_price_diff.csv')

#     absa_df = absa_df.drop(columns=['Date'])
#     print(absa_df.columns[:-3])
    for col in absa_df.columns[:-3]:
        if col == 'Date':
            continue
        absa_df[col] /= absa_df['count']
    absa_df = absa_df.drop(columns=['count'])
    
    return absa_df

In [60]:
def get_absa_no_known_df():
    absa_df = pd.read_csv('./data/processed_absa_price_diff.csv')

#     absa_df = absa_df.drop(columns=['Date'])
    print(absa_df.columns[:-3])
    for col in absa_df.columns[:-3]:
        if col == 'Date':
            continue
        absa_df[col] /= absa_df['count']
    absa_df = absa_df.drop(columns=['count','known'])
    
    return absa_df

In [61]:
def pure_stock_df():
    df = pd.read_csv('./data/processed_sa_bert_price_diff.csv')
    df = df[['Date','known','to_predict']]
    return df

In [7]:
input_dim = 5
hidden_dim = 16
num_layers = 2
output_dim = 1
# Train model
#####################
num_epochs = 40
hist = np.zeros(num_epochs)

# Number of steps to unroll
look_back = 10
seq_dim =look_back-1  

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        x = x.view(len(x), 1, -1)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out
LSTM_model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(LSTM_model.parameters(), lr=0.002)

NameError: name 'np' is not defined

# generalized linear model

## pure stock price

In [511]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Load the data
# df = pure_stock_df()

# Split the data into predictor variables and target variable
# X = df.drop(['to_predict', 'Date'], axis=1).values
# y = df['to_predict'].values

# Normalize the predictor variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the models
models = [
    ('LSTM', LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)),

]

# Define the range of K values
k_values = [10]

# Define a dictionary to store the MSE values for each model and each K value
mse_dict = {model_name: {k: [] for k in k_values} for model_name, _ in models}

# Loop through the K values and perform K-fold cross-validation
for k in k_values:
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train = torch.from_numpy(X_train).type(torch.Tensor)
        X_test = torch.from_numpy(X_test).type(torch.Tensor)
        y_train = torch.from_numpy(y_train).type(torch.Tensor)
        y_test = torch.from_numpy(y_test).type(torch.Tensor)
        # Loop through the models and fit them on the training data
        for model_name, model in models:
            for t in range(num_epochs):
                # Forward pass
                y_train_pred = LSTM_model(X_train)

                loss = loss_fn(y_train_pred, y_train)
                if t % 20 == 0 and t !=0:
                    print("Epoch ", t, "MSE: ", loss.item())
                hist[t] = loss.item()

                # Zero out gradient, else they will accumulate between epochs
                optimiser.zero_grad()

                # Backward pass
                loss.backward()

                # Update parameters
                optimiser.step()
            
            # make predictions
            y_pred = model(X_test)

            # calculate root mean squared error
            mse = mean_squared_error(y_test.detach(), y_pred.detach())
            
            # Store the MSE in the dictionary
            mse_dict[model_name][k].append(mse)


best_mse = {}
for model_name, _ in models:
    best_mse[model_name] = np.min([np.mean(mse_dict[model_name][k]) for k in k_values])
    print(f'{model_name}: {best_mse[model_name]:.4f}')


Epoch  20 MSE:  0.0013035292504355311
Epoch  20 MSE:  0.0011221938766539097
Epoch  20 MSE:  0.0007166484137997031
Epoch  20 MSE:  0.0007366257486864924
Epoch  20 MSE:  0.0005574573297053576
Epoch  20 MSE:  0.00044447719119489193
Epoch  20 MSE:  0.0003679227957036346
Epoch  20 MSE:  0.00035732885589823127
Epoch  20 MSE:  0.0003082369512412697
Epoch  20 MSE:  0.0002783823583740741
LSTM: 0.1008


In [ ]:
# (known -> to_predict): 0.0725-0.0777
# BERT: ('pos', 'neg', 'neu', 'none', 'known')-> ('to_predict') LSTM: 0.0715-0.-0.0722
# RoBERTa: ('pos', 'neg', 'neu', 'none', 'known')-> ('to_predict') LSTM: 0.0711-0.0720